# Business Performance & Customer Insights Dashboard
  ### Project Overview

This dashboard provides a comprehensive analysis of overall business performance using transactional data. It highlights key metrics and trends across:

- Revenue and financial performance

- Sales performance

- Customer Cehavior

- Discount & pricing strategy

The objective is to identify revenue drivers, evaluate customer loyalty, analyze geographic sales distribution, and support data-driven business decisions.

## Importing dataset

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import ipywidgets as widgets
from plotly.subplots import make_subplots
from IPython.display import display, clear_output






amazon = pd.read_csv(
    r"../file/raw data/Amazon.csv"
)

df = amazon.copy()

print(df.columns)
print("Dataset Shape:", df.shape)
print(df.duplicated().sum())
df.info()
df.dtypes
df.isnull().any()               
df.describe()



Index(['OrderID', 'OrderDate', 'CustomerID', 'CustomerName', 'ProductID',
       'ProductName', 'Category', 'Brand', 'Quantity', 'UnitPrice', 'Discount',
       'Tax', 'ShippingCost', 'TotalAmount', 'PaymentMethod', 'OrderStatus',
       'City', 'State', 'Country', 'SellerID'],
      dtype='str')
Dataset Shape: (100000, 20)
0
<class 'pandas.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 20 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   OrderID        100000 non-null  str    
 1   OrderDate      100000 non-null  str    
 2   CustomerID     100000 non-null  str    
 3   CustomerName   100000 non-null  str    
 4   ProductID      100000 non-null  str    
 5   ProductName    100000 non-null  str    
 6   Category       100000 non-null  str    
 7   Brand          100000 non-null  str    
 8   Quantity       100000 non-null  int64  
 9   UnitPrice      100000 non-null  float64
 10  Discount       100000 non-nu

,Quantity,UnitPrice,Discount,Tax,ShippingCost,TotalAmount
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,3.001400,302.905748,0.074226,68.468902,7.406660,918.256479
std,1.413548,171.840797,0.082583,74.131180,4.324057,724.508332
min,1.000000,5.000000,0.000000,0.000000,0.000000,4.270000
25%,2.000000,154.190000,0.000000,15.920000,3.680000,340.890000
50%,3.000000,303.070000,0.050000,45.250000,7.300000,714.315000
75%,4.000000,451.500000,0.100000,96.060000,11.150000,1349.765000
max,5.000000,599.990000,0.300000,538.460000,15.000000,3534.980000


## Data cleaning

In [6]:
# Standardize column names

df.columns = (
    df.columns
    .str.replace(r'([a-z0-9])([A-Z])', r'\1_\2', regex=True)
    .str.replace(r'([A-Z]+)([A-Z][a-z])', r'\1_\2', regex=True)
    .str.lower()
)

df['order_date'] = pd.to_datetime(df['order_date'])
# Standardize text data
df = df.apply(
    lambda col: col.str.lower().str.strip() if col.dtype == "object" else col
)
df.columns
df.head()

,order_id,order_date,customer_id,customer_name,product_id,product_name,category,brand,quantity,unit_price,discount,tax,shipping_cost,total_amount,payment_method,order_status,city,state,country,seller_id
0,ORD0000001,2023-01-31,CUST001504,Vihaan Sharma,P00014,Drone Mini,Books,BrightLux,3,106.59,0.00,0.00,0.09,319.86,Debit Card,Delivered,Washington,DC,India,SELL01967
1,ORD0000002,2023-12-30,CUST000178,Pooja Kumar,P00040,Microphone,Home & Kitchen,UrbanStyle,1,251.37,0.05,19.10,1.74,259.64,Amazon Pay,Delivered,Fort Worth,TX,United States,SELL01298
2,ORD0000003,2022-05-10,CUST047516,Sneha Singh,P00044,Power Bank 20000mAh,Clothing,UrbanStyle,3,35.03,0.10,7.57,5.91,108.06,Debit Card,Delivered,Austin,TX,United States,SELL00908
3,ORD0000004,2023-07-18,CUST030059,Vihaan Reddy,P00041,Webcam Full HD,Home & Kitchen,Zenith,5,33.58,0.15,11.42,5.53,159.66,Cash on Delivery,Delivered,Charlotte,NC,India,SELL01164
4,ORD0000005,2023-02-04,CUST048677,Aditya Kapoor,P00029,T-Shirt,Clothing,KiddoFun,2,515.64,0.25,38.67,9.23,821.36,Credit Card,Cancelled,San Antonio,TX,Canada,SELL01411


In [7]:
# Extract year, month, and month name from OrderDate
df["year"] = df["order_date"].dt.year
df["month"] = df["order_date"].dt.month
df["month_name"] = df["order_date"].dt.month_name()

## Visualization

### Revenue and Financial Performace 

In [10]:

# --- Calculate KPIs ---
total_revenue = df['total_amount'].sum()

# AOV: make sure to calculate correctly per order
order_revenue = df.groupby('order_id')['total_amount'].sum()
aov = order_revenue.mean()

total_orders = df['order_id'].nunique()
total_customers = df['customer_id'].nunique()
total_quantity = df['quantity'].sum()
total_discount = df['discount'].sum()

# --- Create a 2-row, 3-column grid for KPI cards ---
fig = make_subplots(
    rows=2, cols=3,
    specs=[[{"type": "indicator"}, {"type": "indicator"}, {"type": "indicator"}],
           [{"type": "indicator"}, {"type": "indicator"}, {"type": "indicator"}]]
)

# Add KPI cards
fig.add_trace(go.Indicator(
    mode="number",
    value=total_revenue,
    number={'prefix': "$", 'valueformat': ',.0f'},
    title={"text": "Total Revenue"}
), row=1, col=1)

fig.add_trace(go.Indicator(
    mode="number",
    value=aov,
    number={'prefix': "$", 'valueformat': ',.2f'},
    title={"text": "Average Order Value (AOV)"}
), row=1, col=2)

fig.add_trace(go.Indicator(
    mode="number",
    value=total_orders,
    number={'valueformat': ',.0f'},
    title={"text": "Total Orders"}
), row=1, col=3)

fig.add_trace(go.Indicator(
    mode="number",
    value=total_customers,
    number={'valueformat': ',.0f'},
    title={"text": "Total Customers"}
), row=2, col=1)

fig.add_trace(go.Indicator(
    mode="number",
    value=total_quantity,
    number={'valueformat': ',.0f'},
    title={"text": "Total Quantity Sold"}
), row=2, col=2)

fig.add_trace(go.Indicator(
    mode="number",
    value=total_discount,
    number={'prefix': "$", 'valueformat': ',.0f'},
    title={"text": "Total Discount Given"}
), row=2, col=3)

# Layout adjustments
fig.update_layout(
    template='plotly_white',
    height=500,
    title_text=" Key Performance Indicators (KPIs)",
    title_x=0.5
)

fig.show()
fig.write_html("../visuals/Calculate_KPIs.html", full_html=True, include_plotlyjs="cdn")

##### Yearly and Monthly Revenue Trend

In [11]:
# Prepare data
df_monthly = df.groupby('month')['total_amount'].sum().reset_index()
df_yearly = df.groupby('year')['total_amount'].sum().reset_index()

# Create figure
fig = go.Figure()

# ======================
# Monthly Trace
# ======================
fig.add_trace(
    go.Scatter(
        x=df_monthly['month'],
        y=df_monthly['total_amount'],
        mode='lines+markers',
        name='Monthly Revenue',
        visible=True,
        line=dict(shape='spline')
    )
)

# ======================
# Yearly Trace
# ======================
fig.add_trace(
    go.Scatter(
        x=df_yearly['year'],
        y=df_yearly['total_amount'],
        mode='lines+markers',
        name='Yearly Revenue',
        visible=False,
        line=dict(shape='spline')
    )
)

# ======================
# COVID Vertical Lines
# ======================
fig.update_layout(
    shapes=[
        # COVID Start
        dict(
            type="line",
            x0=2020, x1=2020,
            y0=0, y1=1,
            xref="x",
            yref="paper",
            line=dict(dash="dash", width=2),
            visible=False
        ),
        # Post-COVID Recovery
        dict(
            type="line",
            x0=2022, x1=2022,
            y0=0, y1=1,
            xref="x",
            yref="paper",
            line=dict(dash="dash", width=2),
            visible=False
        )
    ]
)

# ======================
# Dropdown Menu
# ======================
fig.update_layout(
    updatemenus=[
        dict(
            buttons=[
                dict(
                    label="Monthly Revenue",
                    method="update",
                    args=[
                        {"visible": [True, False]},  # Show monthly trace
                        {
                            "title": "Monthly Revenue Trend",
                            "shapes[0].visible": False,
                            "shapes[1].visible": False
                        }
                    ],
                ),
                dict(
                    label="Yearly Revenue",
                    method="update",
                    args=[
                        {"visible": [False, True]},  # Show yearly trace
                        {
                            "title": "Yearly Revenue Trend (COVID Impact Highlighted)",
                            "shapes[0].visible": True,
                            "shapes[1].visible": True
                        }
                    ],
                ),
            ],
            direction="down",
            showactive=True,
            x=1,
            xanchor="right",
            y=1.25,
            yanchor="top"
        )
    ]
)

# ======================
# Layout Styling
# ======================
fig.update_layout(
    title="Monthly revenue trend",
    title_x=0.5,
    template="plotly_white",
    margin=dict(t=120)
)

fig.update_xaxes(rangeslider_visible=True)

fig.show()

# Save to HTML
fig.write_html(
    "../visuals/yearly_And_monthly_Revenue_Trend.html",
    full_html=True,
    include_plotlyjs="cdn"
)

##### Top Revenue

In [12]:
dimensions = [
    "category",
    "product_name",
    "brand",
    "seller_id",
    "state",
    "city",
    "payment_method"
]

fig = go.Figure()

total_rev = df['total_amount'].sum()

# Create one trace per dimension
for i, dim in enumerate(dimensions):

    grouped = (
        df.groupby(dim)['total_amount']
        .sum()
        .sort_values(ascending=False)
        .head(15)  # limit for readability
    )

    x_labels = [str(x)[:30] + ("..." if len(str(x)) > 30 else "") for x in grouped.index]

    fig.add_trace(
        go.Bar(
            x=x_labels,
            y=grouped.values,
            visible=True if i == 0 else False,
            customdata=(grouped.values / total_rev) * 100,
            hovertemplate='%{x}<br>Revenue: $%{y:,.0f}<br>Percent of Total: %{customdata:.2f}%'
        )
    )

# Create dropdown buttons
buttons = []

for i, dim in enumerate(dimensions):
    visibility = [False] * len(dimensions)
    visibility[i] = True

    buttons.append(
        dict(
            label=dim.replace('_', ' ').title(),
            method="update",
            args=[
                {"visible": visibility},
                {"title": f"Revenue by {dim.replace('_',' ').title()}"}
            ]
        )
    )

fig.update_layout(
    updatemenus=[
        dict(
            buttons=buttons,
            direction="down",
            showactive=True,
            x=1,
            xanchor="right",
            y=1.2,
            yanchor="top"
        )
    ],
    title="Revenue by category",
    title_x=0.5,
    template="plotly_white",
    margin=dict(t=120),
    xaxis=dict(tickangle=45),
    yaxis=dict(title="Revenue")
)

fig.show()

fig.write_html("../visuals/top_revenue.html")

Revenue vs Tax, Shipping Cost, and Discount

In [13]:

# Aggregate totals
metrics = {
    'Revenue': df['total_amount'].sum(),
    'Tax': df['tax'].sum(),
    'Shipping Cost': df['shipping_cost'].sum(),
    'Discount': df['discount'].sum()
}

# Convert to DataFrame for plotting
metrics_df = pd.DataFrame({
    'Metric': list(metrics.keys()),
    'Amount': list(metrics.values())
})

# Plot bar chart
fig4 = px.bar(
    metrics_df,
    x='Metric',
    y='Amount',
    text='Amount',
    title="Revenue vs Tax, Shipping Cost, and Discount",
    labels={'Amount': 'Amount ($)'}
)

fig4.update_traces(texttemplate='$%{y:,.0f}', textposition='outside')
fig4.update_layout(
    yaxis=dict(range=[0, max(metrics_df['Amount'])*1.1]),
    title_x=0.5,
    template='plotly_white'
)

fig4.show()
fig4.write_html("../visuals/revenue_comparision.html")

### Sales Performance

##### Top 10 Product, Category and Brand

In [14]:
dimensions = [
    "product_name",
    "category",
    "brand"
]

fig = go.Figure()

# Create one treemap per dimension
for i, dim in enumerate(dimensions):

    grouped = (
        df.groupby(dim)['quantity']
        .sum()
        .sort_values(ascending=False)
        .head(10)
        .reset_index()
    )

    fig.add_trace(
        go.Treemap(
            labels=grouped[dim],
            parents=[""] * len(grouped),
            values=grouped['quantity'],
            marker=dict(
                colors=grouped['quantity'],
                colorscale="Blues",
                colorbar=dict(title="Quantity")
            ),
            visible=True if i == 0 else False
        )
    )

# Dropdown buttons
buttons = []

for i, dim in enumerate(dimensions):
    visibility = [False] * len(dimensions)
    visibility[i] = True

    buttons.append(
        dict(
            label=dim.replace('_', ' ').title(),
            method="update",
            args=[
                {"visible": visibility},
                {"title": f"Top 10 {dim.replace('_',' ').title()} by quantity sold"}
            ]
        )
    )

fig.update_layout(
    updatemenus=[
        dict(
            buttons=buttons,
            direction="down",
            showactive=True,
            x=1,
            xanchor="right",
            y=1.15,
            yanchor="top"
        )
    ],
    title="Top 10 Product Names by Quantity Sold",
    title_x=0.5,
    template="plotly_white",
    margin=dict(t=100)
)

fig.show()

fig.write_html(
    "../visuals/top_10_product.html",
    full_html=True,
    include_plotlyjs="cdn"
)

##### Price Sensitivity Analysis

In [15]:
# Create price bins (cleaner bins)
df['price_bin'] = pd.cut(df['unit_price'], bins=12)

# Calculate average quantity per bin
grouped = (
    df.groupby('price_bin', observed=False)['quantity']
      .mean()
      .reset_index()
)

# Convert bin to midpoint for cleaner x-axis
grouped['price_mid'] = grouped['price_bin'].apply(lambda x: x.mid)

# Plot
fig = px.line(
    grouped.sort_values('price_mid'),
    x='price_mid',
    y='quantity',  
    markers=True,
    title="Price Sensitivity Analysis"
)

fig.update_layout(
    title_x=0.5,
    xaxis_title="Price",
    yaxis_title="Average quantity sold",
    template="plotly_white"
)

fig.show()
fig.write_html(
    "../visuals/Price_Sensitivity_Analysis.html")

##### Sales by Location(Quantity)


In [17]:
#sales by location(quantity)
country_revenue = (
    df.groupby('country')['quantity']
    .sum()
    .reset_index()
)

fig6 = px.choropleth(
    country_revenue,
    locations="country",
    locationmode='country names',
    color="quantity",
    hover_name="country",
    color_continuous_scale='Plasma',
    title='Global Revenue by Country'
)
fig6.update_layout(title_x=0.5, template='plotly_white')
fig6.show()
fig6.write_html("../visuals/sales_by_location.html")

C:\Users\User\AppData\Local\Temp\ipykernel_4908\2883734528.py:8: DeprecationWarning:

The library used by the *country names* `locationmode` option is changing in an upcoming version. Country names in existing plots may not work in the new version. To ensure consistent behavior, consider setting `locationmode` to *ISO-3*.



### Customer Behavior

Revenue per Customer

In [18]:

# Revenue per customer
customer_rev = (
    df.groupby('customer_id')['total_amount']
    .sum()
    .sort_values(ascending=False)
    .reset_index()
)

# Calculate cumulative %
customer_rev['cumulative_revenue'] = customer_rev['total_amount'].cumsum()
total_revenue = customer_rev['total_amount'].sum()
customer_rev['cumulative_percent'] = 100 * customer_rev['cumulative_revenue'] / total_revenue

# Customer rank %
customer_rev['customer_rank_percent'] = 100 * (customer_rev.index + 1) / len(customer_rev)

fig7 = px.line(
    customer_rev,
    x='customer_rank_percent',
    y='cumulative_percent',
    title='Revenue Concentration Analysis',
    labels={
        'customer_rank_percent': 'Customer % (Top to Bottom)',
        'cumulative_percent': 'Cumulative Revenue %'
    }
)

fig7.update_layout(
    title_x=0.5,
    template='plotly_white'
)

fig7.show()
fig7.write_html("../visuals/revenue_per_customer.html")

Customer Loyalty

In [19]:
# Orders per customer
orders_per_customer = (
    df.groupby('customer_id')['order_id']
    .nunique()
    .reset_index(name='order_count')
)

fig8 = px.histogram(
    orders_per_customer,
    x='order_count',
    nbins=20,
    title='Customer Purchase Frequency Distribution',
    labels={'order_count': 'Number of Orders per Customer'}
)

fig8.update_layout(
    title_x=0.5,
    template='plotly_white'
)

fig8.show()
fig8.write_html("../visuals/Orders_per_customer.html")

New vs Returning Customers Over Time

In [20]:
df['order_date'] = pd.to_datetime(df['order_date'])

if 'first_purchase_date' in df.columns:
    df = df.drop(columns=['first_purchase_date'])

# First purchase date per customer
first_purchase = (
    df.groupby('customer_id')['order_date']
      .min()
      .reset_index()
      .rename(columns={'order_date': 'first_purchase_date'})
)

# Safe merge
df = df.merge(first_purchase, on='customer_id', how='left')

# Mark new vs returning
df['customer_type'] = np.where(
    df['order_date'] == df['first_purchase_date'],
    'New',
    'Returning'
)

# Monthly aggregation
monthly_customers = (
    df.groupby([pd.Grouper(key='order_date', freq='ME'), 'customer_type'])
    ['customer_id']
    .nunique()
    .reset_index()
)

fig9 = px.line(
    monthly_customers,
    x='order_date',
    y='customer_id',
    color='customer_type',
    title='New vs Returning Customers Over Time',
    labels={'customer_id': 'Number of Customers'}
)

fig9.update_layout(
    title_x=0.5,
    template='plotly_white'
)

fig9.show()
fig9.write_html("../visuals/New_vs_Returning_Customers_Over_Time.html")

### Discount & Pricing Strategy

Discount vs Revenue

In [21]:
discount_revenue = df.groupby('order_id').agg({
    'discount': 'sum',
    'total_amount': 'sum'
}).reset_index()

fig10 = px.scatter(
    discount_revenue,
    x='discount',
    y='total_amount',
    title='Discount vs Revenue per Order',
    labels={'discount': 'Total Discount', 'total_amount': 'Order Revenue'}
)
fig10.update_layout(
    title_x=0.5,
    template='plotly_white'
)
fig10.show()
fig10.write_html("../visuals/Discount_vs_Revenue_per_Order.html")

Revenue: Discounted vs Non-discounted Orders

In [22]:
df['has_discount'] = df['discount'] > 0

revenue_by_discount = df.groupby('has_discount')['total_amount'].sum().reset_index()

fig = px.bar(
    revenue_by_discount,
    x='has_discount',
    y='total_amount',
    title='Revenue: Discounted vs Non-Discounted Orders',
    labels={'has_discount': 'Discount applied', 'total_amount': 'Revenue'}
)
fig.update_layout(
    title_x=0.5,
    template='plotly_white'
)

fig.show()

Average Discount by Category

In [23]:
avg_discount_category = df.groupby('category')['discount'].mean().reset_index()

fig = px.bar(
    avg_discount_category,
    x='category',
    y='discount',
    title='Average Discount by Category'
)
fig.update_layout(
    title_x=0.5,
    template='plotly_white'
)
fig.show()

## Key Insights & Main Findings

The analysis of 100,000 Amazon transactions (2020–2024) reveals consistent patterns in revenue generation, customer behavior, discount effectiveness, and geographic/seller performance. Below are the most important business takeaways derived from the 12 core visualizations.

### Revenue & Financial Performance

- **Total revenue reached $91.83 million** across 100,000 orders, with an **average order value (AOV) of $918.26**.  
  → The business maintains stable average transaction size despite high volume.

- **Monthly revenue is remarkably consistent** - ranging between **$6.93M and $7.89M** (average ~$7.6M/month).  
  → No extreme seasonality visible in monthly aggregates; yearly totals remain stable around **$18.2-18.5M** per year (2020-2024).

- **Revenue distribution by category is nearly uniform** (~16.5-17% each), with **Electronics slightly leading (16.97%)**.  
  → The portfolio is well-balanced. No single category dominates revenue.

- **Credit Card is the dominant payment method** (34.98% of revenue = $32.12M), while **Cash on Delivery contributes the least (4.92%)**.  
  → Digital payment preference supports faster cash flow and lower collection risk.

- **Tax ($6.85M) and shipping cost ($0.74M) represent only ~8.3% of total revenue**, while **discounts given total $7.42M (~8.1%)**.  
  → Margin pressure from discounts is comparable to tax + shipping combined.

### Sales Performance

- **Top products by quantity sold** are mostly everyday electronics/accessories (LED Desk Lamp 6,344 units, Memory Card 128GB, Router, etc.).  
  → High-frequency, low-to-mid price items drive volume.

- **Electronics leads category volume** (50,696 units), followed closely by other categories (~49-50K each).  
  → Volume distribution is balanced, but Electronics has slight edge in units sold.

- **CoreTech is the top-performing brand** (30,430 units), while **HomeEase trails slightly** (29,476 units).  
  → Brand performance is quite even - opportunity to strengthen underperformers.

### Customer Behavior & Loyalty

- **Revenue is highly concentrated**: Top 10% of customers generate ~26-30% of revenue; top 20% generate ~44%; top 40% generate ~70-75%.  
  → Classic Pareto pattern: a small group of high-value customers drives the majority of revenue.

- **Most customers place 1-2 orders** (13,532 and 13,504 customers respectively); only ~1,736 place 5+ orders.  
  → Very low repeat purchase rate - strong indication of weak customer retention/loyalty.

- **Returning customers show clear upward trend** over time (from ~200-500 in early months to 1,200-1,350 in recent months), while new customer acquisition is declining.  
  → Retention is improving slowly, but new customer inflow is shrinking - urgent need for acquisition strategies.

### Discount & Pricing Strategy

- **Discounted orders generate significantly more revenue** ($52.07M) than non-discounted orders ($39.76M).  
  → Discounts are effective in driving overall revenue.

- **Optimal discount range appears to be 5-10%** (highest revenue per order in scatter plot around these levels).  
  → Higher discounts (20-30%) show diminishing or negative returns per order.

- **Average discount rate is very consistent across categories** (~7.4-7.5%), with **Sports & Outdoors slightly higher (7.40%)** and Books/Clothing slightly lower.  
  → No major category-specific discount bias - uniform pricing strategy in place.

### Overall Business Recommendations

1. **Strengthen customer retention**: current repeat rate is very low; implement loyalty programs, personalized offers, and post-purchase engagement to convert one-time buyers into repeat customers.
2. **Protect and grow high-value customer segment**: focus marketing and VIP perks on the top 10-20% who drive disproportionate revenue.
3. **Optimize discount strategy**: maintain 5-10% as sweet spot; avoid deep discounts (>15%) unless for clearance or acquisition.
4. **Continue balanced category approach**: no need for major reallocation; slight push in Electronics could further increase volume.
5. **Improve new customer acquisition**: declining new customer trend is concerning; test acquisition channels (ads, partnerships, promotions).
6. **Monitor payment method mix**: encourage more Credit Card / digital payments to improve cash flow.

These insights provide clear priorities for improving profitability, customer lifetime value, and sustainable growth.